In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score
# import pickle
import glob
import os

---

## fill out the info below

In [2]:
File_path_for_day_data = '/xxx/xxx/code/xxx/daylight image/train/day'
File_path_for_night_data = '/xxx/xxx/code/xxx/daylight image/train/night'
File_path_for_test_day_data = '/xxx/xxx/code/xxx/daylight image/test/day'
File_path_for_test_night_data = '/xxx/xxx/code/xxx/daylight image/test/night'

resize_pixel = 120 # 120x120

# choose the way you want to modify the picture
average_rgb = True
grayscale = False

---

In [3]:
def get_data(img_path):
    folder_path = img_path 
    image_paths = []
    image_paths.extend(glob.glob(os.path.join(folder_path, "*.jpg")))
    return image_paths

In [4]:
def data_clean(img_paths,resize_pixel,average_rgb,grayscale):  # Replace with the path to your folder
#     folder_path = img_path 

#     image_paths = []
#     image_paths.extend(glob.glob(os.path.join(folder_path, "*.jpg")))

    data = []
    x = 0
    print(len(img_paths))
    
    if grayscale == True:
        for i in img_paths:

            image = Image.open(i)
            image = image.rotate(90)
            new_size = (int(resize_pixel), int(resize_pixel))
            image = image.resize((new_size))
            pixel = image.load()
            pic_data = []   
            for colume in range(image.size[1]):
                temp_data = []
                for row in range(image.size[0]):
                    temp_pix = list(pixel[colume,row])

                    # Algorithm to turn picture to grayscale
                    temp_averange = 0.299 * temp_pix[0] + 0.587 * temp_pix[1] + 0.114 * temp_pix[2]

                    temp_data.append(temp_averange)
                pic_data.append(temp_data)
            data.append(pic_data)
            x += 1
            
    elif average_rgb == True:
        for i in img_paths:

            image = Image.open(i)
            image = image.rotate(90)
            image = image.resize((resize_pixel,resize_pixel))
            pixel = image.load()
            pic_data = []   
            for colume in range(image.size[0]):
                temp_data = []
                for row in range(image.size[1]):
                    temp_pix = list(pixel[colume,row])

                    # Algorithm to turn picture to average rgb
                    temp_averange = sum(temp_pix)/3

                    temp_data.append(temp_averange)
                pic_data.append(temp_data)
            data.append(pic_data)
            x += 1
            
    data = np.array(data).reshape((x,int(resize_pixel**2)))
    return data

---


In [5]:
day_data = get_data(File_path_for_day_data)
night_data = get_data(File_path_for_night_data)

In [6]:
df = pd.DataFrame()
df['file'] = day_data
df['mark'] = 1
df = df.reset_index(drop=True)

In [7]:
df1 = pd.DataFrame()
df1['file'] = night_data
df1['mark'] = -1
df1 = df1.reset_index(drop=True)

In [8]:
train_data = pd.concat([df,df1])
train_data =train_data.reset_index(drop=True)

In [9]:
train_data = train_data.sample(frac=1)
train_data =train_data.reset_index(drop=True)

In [10]:
x = data_clean(train_data['file'],resize_pixel,average_rgb,grayscale)
Y = train_data['mark']

0


In [11]:
data = x[0].reshape((resize_pixel, resize_pixel))

# Create a grayscale image using matplotlib
plt.imshow(data, cmap='bone')
plt.axis('off')  # Turn off the axis
plt.show()

IndexError: index 0 is out of bounds for axis 0 with size 0

---

In [ ]:
def supportvectormachine(x, Y, learning_Rate, epochs):
#     archive = []
    x =  np.append(x, np.array([np.ones(len(x))]).T, axis=1)
    weights = np.random.rand(len(x.T)) # One weight for each column of the feature set. (Another way to say it is a weight for each feature.)
    
    # Loop over all the data
    for i in tqdm(range(epochs)):
        
#         # Create an arcive so that we can plot many different SVM lines.
#         archive.append([i,weights[0], weights[1]])
         
        # Predict over all the x feature training data.
        y_pred = np.dot(x, weights)
        
        # Get the value of the true answer times the predicted value. (This is y * <x,w> above)
        value = Y * y_pred
        
        # Set a location. Since we will be working through all the data one at a time, this will keep our position.
        location = 0
        
        # Loop over all the data in the training set. Note in this loop t is set to an element of the array, value.
        for t in value:
            
            # The above sudogradient requires us to check the sign of one part of the gradient. So this gate will do that for us. 
            
            # Correct guess weight update.
            if t > 0:
                weights = weights  # - learning_Rate * (2 * 1 / epochs * weights )
            
            # Incorrect guess weight update.
            else:
                weights  = weights + learning_Rate * ( Y[location] * x[location] ) # - 2 * 1 / epochs * weights)
                
            
            # Move one location in the data set (we will need to do this for each epoch)
            location += 1
    
    # Return the weights, the list of intercepts and the list of slopes.
    return weights

---

In [ ]:
folder_path = File_path_for_test_day_data
image_paths = []
image_paths.extend(glob.glob(os.path.join(folder_path, "*.jpg")))
df2 = pd.DataFrame()
df2['file'] = image_paths
df2['mark'] = 1
df2 = df2.reset_index(drop=True)

In [ ]:
folder_path = File_path_for_test_night_data
image_paths = []
image_paths.extend(glob.glob(os.path.join(folder_path, "*.jpg")))
df3 = pd.DataFrame()
df3['file'] = image_paths
df3['mark'] = -1
df3 = df3.reset_index(drop=True)

In [ ]:
test_data = pd.concat([df2,df3])
test_data =test_data.reset_index(drop=True)
test_data = test_data.sample(frac=1)
test_x = data_clean(test_data['file'],resize_pixel,average_rgb,grayscale)
test_y = test_data['mark']

---


## Train

In [ ]:
for i in range(1):
    
    w = supportvectormachine(x, Y, 0.0001, 100000)

    V = [] # Empty list to collect the predictions.

    test_x = test_x # Assign the testing data.
    test_y = np.array(test_data['mark']) # Assign the testing targets.

    # Loop over all the testing features to get the classification probabilities.
    for i in range(len(test_x)):
        V.append(np.dot(w[:-1], test_x[i]) + w[-1])

    new_y_pred = [] # Empty list for the predictions.

    # If the predicetion value is greater then 1, give it a value of 1 and if it is less than on give it a value of -1.
    for val in V:
        if(val > 0):
            new_y_pred.append(1)
        else:
            new_y_pred.append(-1)    
            
    print("Total number correct: ", accuracy_score(test_y,new_y_pred, normalize = False))
    print("Total number in the testing data: ", len(test_data['mark']))
    accuracy =  accuracy_score(test_y,new_y_pred, normalize = False)/len(test_data['mark'])
    print("score:",accuracy )
        
    if accuracy > 0.94:
        break



In [ ]:
# file_path = "my_list_average_480.pkl"
# with open(file_path,'wb') as file:
#     pickle.dump(w, file)